In [ ]:
import os
import json
import glob
from ultralytics import YOLO
os.environ["PYTORCH_CUDA_ALLOC_CONF"]="expandable_segments:True"

# Initial Training

In [ ]:
# Train the model
os.chdir("/data/mtmoore/school/CSiML_AI395T/final_project/dataset/yolov8_ip8m-h-nw_dataset/")

project="/data/mtmoore/school/CSiML_AI395T/final_project/models/yolo11/iterative"
batchsizes={ "11n": {3840: 2, 2880: 2, 1440: 16, 736: 56}, 
             "11s": {3840: 1, 2880: 2, 1440: 8,  736: 32},
             "11m": {3840: 1, 2880: 1, 1440: 4,  736: 16}, 
             "11l": {3840: 1, 2880: 1, 1440: 3,  736: 12}, 
           }

for model_name in ["11n", "11s"]: # "11n", "11s", "11m" ]:
    for imgsize in [2880]:# [736, 1440, 2880]:
        for i in [150]: #list(range(1, 25)) + list(range(25, 50, 5)) + list(range(60, 101, 10)):
            name = f"{model_name}_{imgsize}imgsz_IP8M-H-NW_{i}epochs"      
            weight_path = os.path.join(project, name, "weights", "epoch")
            completed_epochs = glob.glob(f"{weight_path}*.pt")
            resume_flag = False

            # if we started, see if we finished, if not resume
            if os.path.exists(os.path.join(f"{project}/{name}/weights/last.pt")):
                if len(completed_epochs) >= i:
                    print("We have {completed_epochs} pt files in {project}/{name}/weights, continuing")
                    continue
                    
                print(f"Resuming training from last.pt, {len(completed_epochs)} epochs completed previously")
                model = YOLO(f"{project}/{name}/weights/last.pt") 
                resume_flag = True
                print(f"Resuming {project}/{name} model")

            # starting fresh
            else:
                print(f"Loading pre-trained base model to start {project}/{name}")
                model = YOLO(f"yolo{model_name}.pt")  # load a pretrained model (recommended for training)
            
            results = model.train(resume=resume_flag, data="/data/mtmoore/school/CSiML_AI395T/final_project/dataset/yolov8_ip8m-h-nw_dataset/data.yaml", \
                                  project=project, name=name, epochs=i, imgsz=imgsize, batch=batchsizes[model_name][imgsize], plots=True, val=False, save_period=1, save_txt=True)
